In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Model for testing
model = "Updated"

# Importing the mode shapes and natural frequencies from Abaqus and calculating the modal system matrices
file_path = "./Data/" + model + " model/Deformation/grenland_bridge"
file_path_moment = "./Data/" + model + " model/Moment/grenland_bridge_modes.npy"

# Bridge length vector
x = np.arange(0, 609, 8)

# Mode shape matrix: 3 x n_modes x len(x)
phi = np.load(file_path + "_modes.npy")
phi = phi[:,:,::8]
phi = phi[:,:20,:]
# phi[0,:,:] = 0
# phi[2,:,:] = 0

# Frequency vector: n_modes x 1
f_n = np.load(file_path + "_frequencies.npy")
w_n = f_n * 2 * np.pi

# Number of modes
n_modes = phi.shape[1]

# Damping 
xi = np.load("damping_ratio.npy")
xi_vector = {"Initial": np.array([1, 3, 5, 7, 13, 11, 14, 17, 20, 24, 27, 34, 43, 37, 59]) - 1,
             "Updated": np.array([1, 2, 5, 6, 9, 11, 14, 17, 21, 24, 26, 34, 41, 35, 53]) - 1}                  
xi_interp_length = np.arange(n_modes)
xi_interp = np.interp(xi_interp_length, xi_vector[model], xi)

# Generalized mass matrix
M_gen = np.load(file_path + "_mass.npy") 
M_gen = M_gen[:20]

# Establishing the system matrices
M_modal = np.diag(M_gen)
K_modal = np.zeros((n_modes, n_modes))
C_modal = np.zeros((n_modes, n_modes))

# Calculating the system matrices
for i in range(n_modes):

    K_modal[i, i] = w_n[i]**2 * M_modal[i, i]

    C_modal[i, i] = 2 * M_modal[i, i] * w_n[i] * xi_interp[i]

In [3]:
# Calculating the wind load properties
# NS-EN 1991-1-4

# Mean wind velocity
V_b0 = 23
C_alt = max(1 + (30 - 23)* (55.233 - 900) / (23*(1500-900)) , 1)
C_dir = 1 
C_seas = 1
p = 0.5
C_prob = ((1 - 0.2 * np.log(-np.log(1-p))) / (1 - 0.2 * np.log(-np.log(0.98)))) **0.5
C_0 = 1 
z0 = 0.001
z1 = 10
zmin = 2
L1 = 100
z = 55.233
kr = 0.17
C_r = kr * np.log(z / z0)

V = V_b0 * C_dir* C_seas * C_alt * C_prob * C_0 * C_r

# Wind spectra properties
rho = 1.25
Au = 6.8 
Av = 9.4 
Aw = 9.4 
xLu = L1 * (z / z1)**0.3
xLv = 1 / 4 * xLu
xLw = 1 / 12 * xLu
k_i = 1
Iu = kr * V_b0 * k_i / V
Iv = 3 / 4 * Iu
Iw = 1 / 2 * Iu

# Bridge deck properties
D = 2.6
B = 12
CD_prime = 0
CD_bar = 1.0
CL_prime = 2.22
CL_bar = -0.58
CM_prime = 0.786
CM_bar = -0.017
C_u = 9
C_w = 6 

In [4]:
# Calculating the cross-spectral density of the wind load 

# Buffeting wind load matrices
B_q = np.array([
    [2 * (D / B)* CD_bar, ((D / B) * CD_prime - CL_bar)],
    [2 * CL_bar, (CL_prime + (D / B) * CD_bar)],
    [2 * B * CM_bar, B * CM_prime]
])

C_ae = - rho * V * B / 2 * np.array([
    [2 * (D / B)* CD_bar, ((D / B) * CD_prime - CL_bar), 0],
    [2 * CL_bar, (CL_prime + (D / B) * CD_bar), 0],
    [2 * B * CM_bar, B * CM_prime, 0]
])

K_ae = rho * V**2 * B / 2 * np.array([
    [0, 0, (D / B) * CD_prime],
    [0, 0, CL_prime],
    [0, 0, B * CM_prime]
])

# Frequency vector
w = np.logspace(-3, 1.5, 100)
f = w / (2 * np.pi)

# Wind turbulence 
Su = (Iu*V)**2*Au*xLu/V / ((1+1.5*Au*f*xLu/V)**(5/3)) / (2 * np.pi)
Sw = (Iw*V)**2*Aw*xLw/V / ((1+1.5*Aw*f*xLw/V)**(5/3)) / (2 * np.pi)

# Extending B_q and phi for matrix multiplication
B_q_extended = np.zeros((len(x) * 3, len(x) * 2))

for i in range(len(x)):
    B_q_extended[i*3:(i*3)+3, i*2:(i*2)+2] = B_q

phi_extended = np.zeros((3 * len(x), len(x), n_modes))

for i in range(n_modes):
    for j in range(len(x)):

        phi_extended[j*3:(j*3)+3, j, i] = phi[:, i, j]

In [5]:
# Establishing the spectral density matrices
S_Q = np.zeros((n_modes, n_modes, len(w)), dtype=float)
S_v = np.zeros((len(x) * 2, len(x) * 2))


# Establishing dxdx meshgrid and the integrand matrix
dxdx = np.abs(np.array([x]) - np.array([x]).T)
integrand = np.zeros((len(x), len(x)))

for h in range(len(w)):
    
    # One-point spectra for the wind load
    S_v[0::2, 0::2] = Su[h] * np.exp(-C_u * f[h] * dxdx / V)
    S_v[1::2, 1::2] = Sw[h] * np.exp(-C_w * f[h] * dxdx / V)
    
    S_qq = (rho * V * B / 2)**2 * B_q_extended @ S_v @ B_q_extended.T 
    
    for i in range(n_modes):
        for j in range(n_modes):
            
            integrand = phi_extended[:,:,i].T @ S_qq @ phi_extended[:,:,j]
            
            S_Q[i, j, h] = np.trapz(np.trapz(integrand, x), x)

In [6]:
# First draft for calculating the cross-spectral density of the wind load
# Does the same as the code above, but slower

# integrand = np.zeros((len(x), len(x)))

# S_v = np.zeros((2, 2))
# S_Q = np.zeros((n_modes, n_modes, len(w)), dtype=float)

# for h in range(len(w)):
#     for i in range(n_modes):
#         for j in range(n_modes):
#             for k in range(len(x)):
#                 for l in range(len(x)):

#                     # One-point spectra for the wind load
#                     S_v[0, 0] = Su[h] * np.exp(-C_u * f[h] * np.abs(x[l] - x[k]) / V)
#                     S_v[1, 1] = Sw[h] * np.exp(-C_w * f[h] * np.abs(x[l] - x[k]) / V)
      
#                     S_qq = (rho * V * B / 2)**2 * B_q @ S_v @ B_q.T 

#                     integrand[k, l] = phi[:, i, k].T @ S_qq @ phi[:, j, l] 

#             S_Q[i, j, h] = np.trapz(np.trapz(integrand, x), x) 

In [8]:
# Interpolating the cross-spectral density matrix to get better resolution in the response spectrum

w_interp = np.linspace(0.001, 30, 1000)
f_interp = w_interp / (2 * np.pi)
S_Q_interp = np.zeros((n_modes, n_modes, len(w_interp)))

for i in range(n_modes):
    for j in range(n_modes):

        S_Q_interp[i, j, :] = np.interp(w_interp, w, S_Q[i, j, :])

In [9]:
# Calculating the frequency response function

C_ae_modal = np.zeros((n_modes, n_modes))
K_ae_modal = np.zeros((n_modes, n_modes))
H_modal = np.zeros((n_modes, n_modes, len(w_interp)), dtype=complex)

C_integrand = np.zeros(len(x))
K_integrand = np.zeros(len(x))

for i in range(n_modes):
    for j in range(n_modes):
        for k in range(len(x)):
            C_integrand[k] = phi[:,i,k].T @ C_ae @ phi[:,j,k]
            K_integrand[k] = phi[:,i,k].T @ K_ae @ phi[:,j,k]

        C_ae_modal[i, j] = np.trapz(C_integrand, x)
        K_ae_modal[i, j] = np.trapz(K_integrand, x)

for i in range(len(w_interp)):
    H_modal[:,:,i] = np.linalg.inv((-w_interp[i]**2 * M_modal + 1j * w_interp[i] * (C_modal - C_ae_modal) + K_modal - K_ae_modal))

In [10]:
# Calculating the cross-spectral density of the response based on the DEFORMATION
# phi_r = phi

phi_r = np.zeros((3 * len(x), n_modes))

for i in range(n_modes):
    for j in range(len(x)):
        phi_r[j*3:(j*3)+3, i] = phi[:, i, j]

# Cross-spectral density of the response
S_rr = np.zeros((3 * len(x), 3 * len(x), len(w_interp)), dtype=complex)

for i in range(len(w_interp)):
    S_rr[:,:,i] = phi_r @ (np.conj(H_modal[:,:,i]) @ S_Q_interp[:,:,i] @ H_modal[:,:,i].T) @ phi_r.T